In [22]:
from datasets import load_dataset, DatasetDict

lince_ner = load_dataset("lince", "ner_spaeng")

lince_ner = DatasetDict(
    train=lince_ner["train"],
    validation=lince_ner["validation"],
)

Downloading:   0%|          | 0.00/820k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset lince downloaded and prepared to /home/jmperez/.cache/huggingface/datasets/lince/pos_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [25]:
lince_ner["test"][0]

{'idx': 0,
 'words': ['they', "'re", 'from', 'Bakers', '?'],
 'pos': ['', '', '', '', ''],
 'lid': ['eng', 'eng', 'eng', 'eng&spa', 'eng&spa']}

In [12]:
{x for l in lince_ner["train"]["ner"] for x in l}

{'B-EVENT',
 'B-GROUP',
 'B-LOC',
 'B-ORG',
 'B-OTHER',
 'B-PER',
 'B-PROD',
 'B-TIME',
 'B-TITLE',
 'I-EVENT',
 'I-GROUP',
 'I-LOC',
 'I-ORG',
 'I-OTHER',
 'I-PER',
 'I-PROD',
 'I-TIME',
 'I-TITLE',
 'O'}

In [13]:
id2label = [
    'O',
    'B-EVENT',
    'I-EVENT',
    'B-GROUP',
    'I-GROUP',
    'B-LOC',
    'I-LOC',
    'B-ORG',
    'I-ORG',
    'B-OTHER',
    'I-OTHER',
    'B-PER',
    'I-PER',
    'B-PROD',
    'I-PROD',
    'B-TIME',
    'I-TIME',
    'B-TITLE',
    'I-TITLE',
]

label2id = {v:k for k,v in enumerate(id2label)}


In [14]:
lince_ner = lince_ner.map(lambda x:  {"labels": [label2id[l] for l in x["ner"]]})

  0%|          | 0/33611 [00:00<?, ?ex/s]

  0%|          | 0/10085 [00:00<?, ?ex/s]

In [15]:
idx = 11065
print(lince_ner["train"][idx]["ner"])
print(lince_ner["train"][idx]["labels"])
lince_ner["train"][idx].keys(), lince_ner["train"][idx]

['O', 'O', 'O', 'B-OTHER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O']
[0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 11, 12, 0, 11, 0, 0, 0, 0, 0]


(dict_keys(['idx', 'words', 'ner', 'lid', 'labels']),
 {'idx': 11065,
  'words': ['Im',
   'stuck',
   'in',
   'bachata',
   'for',
   'the',
   'rest',
   'of',
   'the',
   'week',
   '...',
   'prince',
   'royce',
   'and',
   'romeo',
   'are',
   'killing',
   'it',
   '!',
   ':)'],
  'ner': ['O',
   'O',
   'O',
   'B-OTHER',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'B-PER',
   'I-PER',
   'O',
   'B-PER',
   'O',
   'O',
   'O',
   'O',
   'O'],
  'lid': ['lang1',
   'lang1',
   'lang1',
   'lang2',
   'lang1',
   'lang1',
   'lang1',
   'lang1',
   'lang1',
   'lang1',
   'other',
   'ne',
   'ne',
   'lang1',
   'ne',
   'lang1',
   'lang1',
   'lang1',
   'other',
   'other'],
  'labels': [0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 11, 12, 0, 11, 0, 0, 0, 0, 0]})

In [16]:
len(lince_ner["train"])

33611

In [17]:
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer


device = "cuda" if torch.cuda.is_available() else "cpu"
max_length = 128

model_name = "pysentimiento/robertuito-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [18]:
from pysentimiento.preprocessing import preprocess_tweet

def preprocess_token(t, lang="es"):
    """
    Seguro podemos hacerlo mejor
    """
    return preprocess_tweet(t, lang="es", demoji=False, preprocess_hashtags=False)

words = lince_ner["train"][idx]["words"]

preprocessed_tweet = [preprocess_token(t) for t in words]

tokens = tokenizer(preprocessed_tweet, is_split_into_words=True)

inv_vocab = {v:k for k, v in tokenizer.get_vocab().items()}

print(words)
list(zip(tokens.word_ids(), [inv_vocab[t] for t in tokens["input_ids"]]))

['Im', 'stuck', 'in', 'bachata', 'for', 'the', 'rest', 'of', 'the', 'week', '...', 'prince', 'royce', 'and', 'romeo', 'are', 'killing', 'it', '!', ':)']


[(None, '<s>'),
 (0, '▁im'),
 (1, '▁stu'),
 (1, 'ck'),
 (2, '▁in'),
 (3, '▁bacha'),
 (3, 'ta'),
 (4, '▁for'),
 (5, '▁the'),
 (6, '▁rest'),
 (7, '▁of'),
 (8, '▁the'),
 (9, '▁week'),
 (10, '▁...'),
 (11, '▁prince'),
 (12, '▁roy'),
 (12, 'ce'),
 (13, '▁and'),
 (14, '▁romeo'),
 (15, '▁are'),
 (16, '▁killing'),
 (17, '▁it'),
 (18, '▁!'),
 (19, '▁:)'),
 (None, '</s>')]

In [124]:
def align_labels_with_tokens(labels, word_ids):
    """
    Tomado de https://huggingface.co/course/chapter7/2?fw=pt
    """
    old_labels = labels
    labels = [label2id[l] for l in labels]
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [125]:
words = lince_ner["train"][idx]["words"]
labels = lince_ner["train"][idx][""]
preprocessed_tweet = [preprocess_token(t) for t in words]

tokens = tokenizer(preprocessed_tweet, is_split_into_words=True)

tokens.word_ids()


[None,
 0,
 1,
 1,
 2,
 3,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 None]